In [ ]:
import os
from coffea import util
import uproot
import numpy as np
import mplhep as hep
import hist
from hist import Hist
from matplotlib import pyplot as plt
from cycler import cycler
from hist.axis import Variable

def get_empty_hist():
    return Hist(
        Variable(
            [
                200,
                220,
                240,
                260,
                280,
                300,
                320,
                340,
                360,
                380,
                400,
                450,
                550,
                700,
                1000,
                2400,
            ],
            name="mass",
        )
    )

year = "2017"
mc = util.load(f'../output/MC_UL_{year}_all_OS.coffea')
signal = util.load(f'../output/signal_UL_{year}_all_OS.coffea')
data = util.load(f'../output/data_UL_{year}_OS_ub.coffea')

group_labels = { 
    "2018": {
        "TT": [
            "TTToSemiLeptonic",
            "TTToHadronic",
            "TTTo2L2Nu",
        ],
        "TTZ": [
            "ttZJets",
        ],    
        "TTW": [
            "TTWJetsToLNu",
        ],
        "ggZZ": [
            "GluGluToContinToZZTo2e2tau",
            "GluGluToContinToZZTo2mu2tau",
            "GluGluToContinToZZTo4e",
            "GluGluToContinToZZTo4mu",
            "GluGluToContinToZZTo4tau",
        ],
        "ZZ": [
            "ZZTo4L",
            "ZZTo2Q2Lmllmin4p0",
        ],
        "WZ": [
            "WZTo2Q2Lmllmin4p0",
            "WZTo3LNu",
        ],
        "VVV": [
            "WWW4F",
            "WWW4F_ext1",
            "WWZ4F",
            "WZZ",
            "WZZTuneCP5_ext1",
            "ZZZ",
            "ZZZTuneCP5_ext1",
        ],
        "ggHtt": [
            "GluGluHToTauTauM125",
        ],
        "VBFHtt": [
            "VBFHToTauTauM125",
        ],
        "WHtt": [
            "WminusHToTauTauM125",
            "WplusHToTauTauM125",
        ],
        "ZHtt": [
            "ZHToTauTauM125_ext1",
        ],
        "TTHtt": [
            "ttHToTauTauM125",
        ],
        "ggHWW": [
            "GluGluHToWWTo2L2NuM-125",
        ],
        "VBFHWW": [
            "VBFHToWWTo2L2NuM-125",
        ], 
        "ggZHWW": [
            "GluGluZHHToWW",
        ],
        "ggHZZ": [
            "GluGluHToZZTo4LM125",
        ],
        "WHWW": [
            "HWminusJHToWW",
            "HWplusJHToWWTo2L2Nu",
        ],
        "ZHWW": [
            "HZJHToWW",
        ]
    },
    "2017": {
        "TT": [
            "TTToSemiLeptonicTuneCP5",
            "TTToHadronicTuneCP5",
            "TTTo2L2NuTuneCP5",
        ],
        "TTZ": [
            "ttZJetsTuneCP5",
        ],    
        "TTW": [
            "TTWJetsToLNuTuneCP5",
        ],
        "ggZZ": [
            "GluGluToContinToZZTo2e2tauTuneCP5",
            "GluGluToContinToZZTo2mu2tauTuneCP5",
            "GluGluToContinToZZTo4eTuneCP5",
            "GluGluToContinToZZTo4muTuneCP5",
            "GluGluToContinToZZTo4tauTuneCP5",
        ],
        "ZZ": [
            "ZZTo4LTuneCP5",
            "ZZTo2Q2Lmllmin4p0",
        ],
        "WZ": [
            "WZTo2Q2Lmllmin4p0",
            "WZTo3LNuTuneCP5",
        ],
        "VVV": [
            "WWW4F",
            "WWW4F_ext1",
            "WWZ4F",
            #"WZZTuneCP5",
            "WZZTuneCP5_ext1",
            "ZZZTuneCP5",
            #"ZZZTuneCP5_ext1",
        ],
        "ggHtt": [
            "GluGluHToTauTauM125",
        ],
        "VBFHtt": [
            "VBFHToTauTauM125",
        ],
        "WHtt": [
            "WminusHToTauTauM125",
            "WplusHToTauTauM125",
        ],
        "ZHtt": [
            "ZHToTauTauM125_ext1",
        ],
        "TTHtt": [
            "ttHToTauTauM125",
        ],
        "ggHWW": [
            "GluGluHToWWTo2L2NuM-125",
        ],
        "VBFHWW": [
            "VBFHToWWTo2L2NuM-125",
        ], 
        "ggZHWW": [
            "GluGluZHHToWW",
        ],
        "ggHZZ": [
            "GluGluHToZZTo4LM125",
        ],
        "WHWW": [
            "HWminusJHToWW",
            "HWplusJHToWW",
        ],
        "ZHWW": [
            "HZJHToWWTo2L2Nu",
        ]
    }
}

cats = ["eeet", "eemt", "eett", "eeem", "mmet", "mmmt", "mmtt", "mmem"]
systs = [
    "nom",
    "l1prefire_up",
    "l1prefire_down",
    "pileup_up",
    "pileup_down",
    "tauES_down",
    "tauES_up",
    "efake_down",
    "efake_up",
    "mfake_down",
    "mfake_up",
    "eleES_down",
    "eleES_up",
    "eleSmear_down",
    "eleSmear_up",
    "muES_down",
    "muES_up",
    "unclMET_down",
    "unclMET_up",
]

In [ ]:
for b in [0, 1]:
    btag_label = "btag" if (b==1) else "0btag"
    
    # fill MC output ROOT file
    mc_file = uproot.recreate(f"root_for_combine/MC_{btag_label}_{year}.root")
    for group, datasets in group_labels[year].items():
        mc_group = sum(
            v for k, v in mc["m4l"].items()
            if k in datasets #k.strip("TuneCP5")
        )
        
        found = [k for k, _ in mc['m4l'].items() if k in datasets]
        print(f"{group}: found {found}")
        assert len(found)==len(datasets)
        
        for cat in cats:
            for syst in systs:         
                if ("btag" in syst): continue
                if (
                    (cat not in list(mc_group.axes[1])) or
                    (syst not in list(mc_group.axes[3]))
                   ):
                    group_hist = get_empty_hist()
                else: 
                    group_hist = mc_group[::sum, cat, b, syst, "cons", :]

                if "nom" in syst:
                    fname = f"{cat}/{group}"
                    mc_file[fname] =  group_hist.to_numpy()
                else:
                    shift = syst.split("_")[-1]
                    syst = syst.replace(f"_{shift}", "")
                    syst = syst + shift.capitalize()
                    fname = f"{cat}/{group}_{syst}"
                    mc_file[fname] = group_hist.to_numpy()
    
    data_group = sum(
        v for k, v in data["m4l"].items()
    )   
    
    # fill reducible into the MC file
    for cat in cats:
        if (cat not in list(data_group.axes[1])):
            group_hist = get_empty_hist()
        else: 
            group_hist = data_group["reducible", cat, b, ::sum, "cons", :]
        fname = f"{cat}/reducible"
        mc_file[fname] = group_hist.to_numpy()
    
    # now fill the fill data
    data_file = uproot.recreate(f"root_for_combine/data_{year}_{btag_label}.root")
    data_group = sum(
        v for k, v in data["m4l"].items()
    )  
    print([k for k in data["m4l"].keys()])
    for cat in cats:
        if (cat not in list(data_group.axes[1])):
            group_hist = get_empty_hist()
        else: 
            group_hist = data_group["data", cat, b, ::sum, "cons", :]

        fname = f"{cat}/data"
        data_file[fname] = group_hist.to_numpy()
    
    # now fill for signal
    for k, v in signal["m4l"].items():
        if (b==0) and "GluGlu" not in k: continue
        if (b>0) and "BBA" not in k: continue
        label = "ggA" if b==0 else "bbA"        
        k = label + f"_{k.split('TauM')[-1]}"
        print(k)
        file = uproot.recreate(f"root_for_combine/{k}_{year}.root")
        for cat in cats:
            for syst in systs:                    
                if ("btag" in syst): continue
                if (
                    (cat not in list(v.axes[1])) or
                    (syst not in list(v.axes[3])) 
                   ):
                    group_hist = get_empty_hist()
                else:
                    group_hist = v[::sum, cat, b, syst, "cons", :]
                    
                if "nom" in syst:
                    fname = f"{cat}/{k}"
                    file[fname] =  group_hist.to_numpy()
                else:
                    shift = syst.split("_")[-1]
                    syst = syst.replace(f"_{shift}", "")
                    syst = syst + shift.capitalize()
                    fname = f"{cat}/{k}_{syst}"
                    file[fname] = group_hist.to_numpy()
5

In [ ]:
uproot.open(
    "/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2017/BBAToZhToLLTauTauM1400/all_BBAToZhToLLTauTau_M1400_file001_part_1of3_Electrons.root"
)

In [ ]:
uproot.open(
    '~/nobackup/combine/CMSSW_11_3_4/src/HiggsAnalysis/CombinedLimit/data/tutorials/longexercise/datacard_part3.shapes.root'
).keys()

In [ ]:
import uproot
base = "/eos/uscms/store/group/lpcsusyhiggs/ntuples/AZh/nAODv9/2017/BBAToZhToLLTauTauM1400/all_BBAToZhToLLTauTau_M1400_file001_part_1of3_Electrons.root"
uproot.open(base)["Events"].arrays()